In [90]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
%config Completer.use_jedi = True
from sklearn.impute import SimpleImputer

In [91]:
untouched_df = pd.read_excel('data.xlsx')

In [504]:
df = untouched_df

In [505]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64678 entries, 0 to 64677
Data columns (total 42 columns):
 #   Column                                                Non-Null Count  Dtype         
---  ------                                                --------------  -----         
 0   ID проекта                                            64678 non-null  object        
 1   ID корпуса                                            64678 non-null  object        
 2   Проект                                                64678 non-null  object        
 3   Локация                                               64678 non-null  object        
 4   Округ                                                 64675 non-null  object        
 5   Район                                                 64678 non-null  object        
 6   Адрес корпуса                                         64678 non-null  object        
 7   Стадия строительной готовности на дату договора       64678 non-null  object

In [506]:
df = df.drop(['ID проекта', 'ID корпуса', 'Локация', 'ID лота', 'Расчетный бюджет объекта', 'Кол-во месяцев обременения', 'Рост бюджета покупки за период экспонирования', 'Округ', 'Месяц и год даты договора','Месяц и год даты регистрации'], axis=1)

In [507]:
df = df.rename(columns={'Количество комнат в прайс-листе, типология bnmap.pro': 'Количество комнат'})

In [508]:
df.head()

,Проект,Район,Адрес корпуса,Стадия строительной готовности на дату договора,Заявленный срок ввода в эксплуатацию,Старт продаж,Класс,Девелопер,Дата договора,Квартал и год договора,Год договора,Дата регистрации,Квартал и год регистрации,Год регистрации,Тип сделки,Тип объекта,Секция,Этаж,Количество комнат,Площадь согласно ПД,Цена за кв. метр,Метод определения цены,Включение скидок/акций в ценообразование,Скидки по дате договора,Отделка по корпусу,Тип продавца,Тип покупателя,Кол-во покупателей,Тип ипотеки,Участие объекта в оптовой сделке,"Срок в экспозиции до момента сделки, дней",Рост цены за 1 кв.м за период экспонирования
0,Баркли Медовая Долина,НАО,"Крекшино, д. 9 (3 оч.)",Строительство не начато,2020-06-30,2017-05-01,Комфорт,Barkli,2019-01-01,1кв. 2019,2019,2019-02-18,1кв. 2019,2019,Договор участия,квартира,NaN,5,ст,27.43,95325.34,Прайс совпадение площади,без скидки,Скидка 12% на ограниченный пул квартир. Скидка...,С отделкой,ЮЛ,ФЛ,1,Нет,Нет,н/д,н/д
1,Румянцево-Парк,НАО,"Московский п., в районе Румянцево дер., уч. 17...",Монтажные и отделочные работы,2020-06-30,2018-04-01,Комфорт,Lexion Development,2019-01-01,1кв. 2019,2019,2019-05-08,2кв. 2019,2019,Договор участия,квартира,4.0,11,2,62.60,114000.00,Прайс совпадение площади,без скидки,Скидка 2% при 100% оплате или ипотеке. При еди...,Без отделки,ЮЛ,ФЛ,1,Нет,Нет,211,"7,5%"
2,Румянцево-Парк,НАО,"Московский п., в районе Румянцево дер., уч. 17...",Монтажные и отделочные работы,2020-06-30,2018-04-01,Комфорт,Lexion Development,2019-01-01,1кв. 2019,2019,2019-05-08,2кв. 2019,2019,Договор участия,квартира,13.0,22,ст,24.10,143000.00,Прайс совпадение площади,без скидки,Скидка 2% при 100% оплате или ипотеке. При еди...,Без отделки,ЮЛ,ФЛ,3,Нет,Нет,211,"16,3%"
3,Баркли Медовая Долина,НАО,"Крекшино, д. 5 (2 оч.)",Монтажные и отделочные работы,2019-09-30,2016-08-01,Комфорт,Barkli,2019-01-02,1кв. 2019,2019,2019-10-14,4кв. 2019,2019,Договор уступки,квартира,NaN,6,ст,28.22,102625.78,Прайс совпадение площади,без скидки,Скидка 12% на ограниченный пул квартир. Скидка...,Без отделки,ЮЛ,ФЛ,1,Ипотека,Нет,н/д,н/д
4,Саларьево Парк,НАО,"Московский п., в районе дер. Саларьево, к. 15",Монтажные и отделочные работы,2020-03-31,2018-02-01,Комфорт,ПИК,2019-01-03,1кв. 2019,2019,2019-02-13,1кв. 2019,2019,Договор участия,квартира,1.0,21,2,51.20,131700.00,Прайс совпадение площади,без скидки,Скидка до 2% при 100% оплате или ипотеке (дейс...,С отделкой,ЮЛ,ФЛ,1,Нет,Нет,275,"15,0%"


In [509]:
label_dict = {}

def label_encode(name):
    df[name] = pd.Categorical(df[name])
    d = dict(enumerate(df[name].cat.categories))
    inverted_d = {v: k for k, v in d.items()}
    print(name, inverted_d.keys())
    df[name] = df[name].cat.codes
    label_dict.update(inverted_d)

In [510]:
df['Стадия строительной готовности на дату договора'].describe()

count                             64678
unique                                7
top       Монтажные и отделочные работы
freq                              35493
Name: Стадия строительной готовности на дату договора, dtype: object

In [511]:
df['Заявленный срок ввода в эксплуатацию'] = pd.to_datetime(df['Заявленный срок ввода в эксплуатацию'], errors='coerce')

In [512]:
df = df.drop('Год договора', axis=1)
df = df.drop('Год регистрации', axis=1)

In [513]:
def split_quater(name):
    quater,year = [],[]
    for i in df[name]:
        quater.append(i.split(' ')[0][0])
        year.append(i.split(' ')[-1])
    return quater, year

In [514]:
df.insert(9, "Квартал договора", split_quater('Квартал и год договора')[0])
df = df.drop('Квартал и год договора', axis=1)
df['Квартал договора'] = df['Квартал договора'].astype(str).astype(int)
df['Дата договора'] = pd.to_datetime(df['Дата договора'])

In [515]:
df.insert(12, "Квартал регистрации", split_quater('Квартал и год регистрации')[0])
df = df.drop('Квартал и год регистрации', axis=1)
df['Квартал регистрации'] = df['Квартал регистрации'].astype(str).astype(int)

In [516]:
df['Секция']=df['Секция'].replace(float('nan'),0)
imputer = SimpleImputer(missing_values=0, strategy='mean')
df['Секция'] = imputer.fit_transform(df['Секция'].values.reshape(-1, 1))
df['Секция'] = df['Секция'].astype(int)

In [517]:
df = df.dropna(subset = ['Цена за кв. метр'])

In [518]:
df['Количество комнат'] = df['Количество комнат'].replace('ст','студия')
df['Количество комнат'] = df['Количество комнат'].replace(float('nan'),0)
print(df['Количество комнат'].unique())
df = df[df['Количество комнат'] != 0]
df = df[df['Количество комнат'] != 'bez-tipa']
df = df[df['Количество комнат'] != '-']
print(df['Количество комнат'].unique())

['студия' '2' '1' '3' 0 '4' '-']
['студия' '2' '1' '3' '4']


In [519]:
# df.to_excel('map.xlsx')

## Transforming for ML

In [520]:
df.head(5)

,Проект,Район,Адрес корпуса,Стадия строительной готовности на дату договора,Заявленный срок ввода в эксплуатацию,Старт продаж,Класс,Девелопер,Дата договора,Квартал договора,Дата регистрации,Квартал регистрации,Тип сделки,Тип объекта,Секция,Этаж,Количество комнат,Площадь согласно ПД,Цена за кв. метр,Метод определения цены,Включение скидок/акций в ценообразование,Скидки по дате договора,Отделка по корпусу,Тип продавца,Тип покупателя,Кол-во покупателей,Тип ипотеки,Участие объекта в оптовой сделке,"Срок в экспозиции до момента сделки, дней",Рост цены за 1 кв.м за период экспонирования
0,Баркли Медовая Долина,НАО,"Крекшино, д. 9 (3 оч.)",Строительство не начато,2020-06-30,2017-05-01,Комфорт,Barkli,2019-01-01,1,2019-02-18,1,Договор участия,квартира,4,5,студия,27.43,95325.34,Прайс совпадение площади,без скидки,Скидка 12% на ограниченный пул квартир. Скидка...,С отделкой,ЮЛ,ФЛ,1,Нет,Нет,н/д,н/д
1,Румянцево-Парк,НАО,"Московский п., в районе Румянцево дер., уч. 17...",Монтажные и отделочные работы,2020-06-30,2018-04-01,Комфорт,Lexion Development,2019-01-01,1,2019-05-08,2,Договор участия,квартира,4,11,2,62.60,114000.00,Прайс совпадение площади,без скидки,Скидка 2% при 100% оплате или ипотеке. При еди...,Без отделки,ЮЛ,ФЛ,1,Нет,Нет,211,"7,5%"
2,Румянцево-Парк,НАО,"Московский п., в районе Румянцево дер., уч. 17...",Монтажные и отделочные работы,2020-06-30,2018-04-01,Комфорт,Lexion Development,2019-01-01,1,2019-05-08,2,Договор участия,квартира,13,22,студия,24.10,143000.00,Прайс совпадение площади,без скидки,Скидка 2% при 100% оплате или ипотеке. При еди...,Без отделки,ЮЛ,ФЛ,3,Нет,Нет,211,"16,3%"
3,Баркли Медовая Долина,НАО,"Крекшино, д. 5 (2 оч.)",Монтажные и отделочные работы,2019-09-30,2016-08-01,Комфорт,Barkli,2019-01-02,1,2019-10-14,4,Договор уступки,квартира,4,6,студия,28.22,102625.78,Прайс совпадение площади,без скидки,Скидка 12% на ограниченный пул квартир. Скидка...,Без отделки,ЮЛ,ФЛ,1,Ипотека,Нет,н/д,н/д
4,Саларьево Парк,НАО,"Московский п., в районе дер. Саларьево, к. 15",Монтажные и отделочные работы,2020-03-31,2018-02-01,Комфорт,ПИК,2019-01-03,1,2019-02-13,1,Договор участия,квартира,1,21,2,51.20,131700.00,Прайс совпадение площади,без скидки,Скидка до 2% при 100% оплате или ипотеке (дейс...,С отделкой,ЮЛ,ФЛ,1,Нет,Нет,275,"15,0%"


In [521]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63322 entries, 0 to 64677
Data columns (total 30 columns):
 #   Column                                           Non-Null Count  Dtype         
---  ------                                           --------------  -----         
 0   Проект                                           63322 non-null  object        
 1   Район                                            63322 non-null  object        
 2   Адрес корпуса                                    63322 non-null  object        
 3   Стадия строительной готовности на дату договора  63322 non-null  object        
 4   Заявленный срок ввода в эксплуатацию             62937 non-null  datetime64[ns]
 5   Старт продаж                                     63322 non-null  datetime64[ns]
 6   Класс                                            63322 non-null  object        
 7   Девелопер                                        63322 non-null  object        
 8   Дата договора                       

In [522]:
label_encode('Район')
label_encode('Тип сделки')
label_encode('Тип объекта')
label_encode('Метод определения цены')
label_encode('Включение скидок/акций в ценообразование')
label_encode('Тип продавца')
label_encode('Тип покупателя')
label_encode('Тип ипотеки')
label_encode('Участие объекта в оптовой сделке')
label_encode('Стадия строительной готовности на дату договора')
label_encode('Класс')
label_encode('Девелопер')
label_encode('Отделка по корпусу')

Район dict_keys(['НАО', 'ТАО'])
Тип сделки dict_keys(['Договор уступки', 'Договор участия'])
Тип объекта dict_keys(['апартаменты', 'квартира'])
Метод определения цены dict_keys(['Аналитика по корпусу', 'Аналитика по типу комнатности', 'Крайние значения обновлений', 'Первая публичная цена старше 2х месяцев', 'Прайс совпадение площади', 'Прайс точное совпадение', 'Цена по последнему историческому слою', 'Цена позже даты сделки'])
Включение скидок/акций в ценообразование dict_keys(['без скидки', 'со скидкой', 'со скидкой и без скидки'])
Тип продавца dict_keys(['ФЛ', 'ЮЛ'])
Тип покупателя dict_keys(['ФЛ', 'ЮЛ'])
Тип ипотеки dict_keys(['Ипотека', 'Нет'])
Участие объекта в оптовой сделке dict_keys(['ДА', 'Нет'])
Стадия строительной готовности на дату договора dict_keys(['Введен в эксплуатацию', 'Монтажные и отделочные работы', 'На дату договора активности по проекту не фиксировалось', 'Начало монтажных работ', 'Получение РВЭ, благоустройство территории', 'Работы нулевого цикла', 'Строительст

In [523]:
df['Количество комнат'] = df['Количество комнат'].replace('студия',0)
df['Количество комнат'] = df['Количество комнат'].astype(str).astype(int)

In [524]:
df = df.drop('Проект', axis=1)
df = df.drop('Адрес корпуса', axis=1)

In [525]:
# max может влиять на модель, проверить с min
import re
c = 0
discounts = []
df['Скидки по дате договора'] = df['Скидки по дате договора'].replace(float('nan'),'')
for i in df['Скидки по дате договора']:
    found = "".join(re.findall(r'Скидка \d+%|Скидка до \d+%',i))
    if len(found) != 0:
        max_discount = max([int(i) for i in re.findall('[0-9]+',found)])
        discounts.append(max_discount)
    else:
        discounts.append(0)
        

In [526]:
df['Скидки по дате договора'] = discounts

In [527]:
# н/д в 0 может повлиять на модель!

df['Срок в экспозиции до момента сделки, дней'] = df['Срок в экспозиции до момента сделки, дней'].replace('н/д', 0)
df['Срок в экспозиции до момента сделки, дней'] = df['Срок в экспозиции до момента сделки, дней'].astype(str).astype(int)
df['Срок в экспозиции до момента сделки, дней'] = df['Срок в экспозиции до момента сделки, дней'].abs()

In [528]:
price = df['Рост цены за 1 кв.м за период экспонирования']
price = [float(i.split('%')[0].replace(',','.')) if '%' in i else i for i in price]
df['Рост цены за 1 кв.м за период экспонирования'] = price
df['Рост цены за 1 кв.м за период экспонирования'] = df['Рост цены за 1 кв.м за период экспонирования'].replace('н/д', 0)

In [529]:
df['Старт продаж'][0]

Timestamp('2017-05-01 00:00:00')

In [530]:
df['Заявленный срок ввода в эксплуатацию'][0]

Timestamp('2020-06-30 00:00:00')

In [531]:

df.insert(4, 'Срок ввода в эксплуатацию - Старт продаж' ,(df['Заявленный срок ввода в эксплуатацию'] - df['Старт продаж']).dt.days)


df.insert(8, 'Срок ввода в эксплуатацию - Договор' ,(df['Заявленный срок ввода в эксплуатацию']-df['Дата договора']).dt.days)


df = df.drop(['Заявленный срок ввода в эксплуатацию','Дата договора', 'Старт продаж', 'Дата регистрации'], axis=1)

In [532]:
# print(label_dict)

In [534]:
df['Срок в экспозиции до момента сделки, дней'].mean()

96.56732257351315

In [535]:
df['Срок ввода в эксплуатацию - Старт продаж'].mean()

789.7537537537537

In [536]:
df['Срок ввода в эксплуатацию - Договор'].mean()

492.6084020528465

In [537]:
df['Площадь согласно ПД'].mean()

49.47615962856511

In [538]:
df['Рост цены за 1 кв.м за период экспонирования'].mean()

5.041318972868829